# Signal Detection Using Templates and Statistics
Elizabeth Kapelevich

In [1]:
import numpy as np
import math
from scipy import interpolate

## Part I: PPT Slides
* Gravity
* General Relativity
* Gravitational Waves

## Part II: Demo
* Write a function returning `t, y` of a sine-gaussian wave
* Plot the default sine-gaussian wave
* Plot different sine-gaussian waves with different values of `a, f,` and $\sigma$ so audience understands how parameters affect wave
* Generate random noise
* Plot the random noise
* Plot the wave on top of the random noise and make the wave an obvious color so audience can see it
* Remove the obvious color so the wave is more difficult to find for the audience
* Show how increasing the amplitude of the noise can completely bury the wave so it's not visible
* Create the `integrator` and `cross_correlation` function
* Make an animation of the `cross_correlation` function sliding with the template across the noise to find the wave
* Plot the `cross_correlation` result and show that the peak is where the wave is found
* Show how changing parameters will affect the peak of `cross_correlation`

In [2]:
def sine_gaussian(amplitude, frequency, sigma, time_interval, t_start, t_end):
    
    """
    This function takes as input the amplitude,
    frequency, standard deviation, interval of
    time, and the bounds of time.
    
    INPUT:
    ------
    amplitude : amplitude
    frequency : frequency
    sigma : standard deviation
    time_interval : interval of time between each value
    t_start : initial time
    t_end : final time
    
    RETURNS:
    --------
    The time series for a wave of a sine-gaussian 
    function.
    """
    
    t_duration = t_end - t_start                                   # calculating duration of time
    time_stamps = t_duration / time_interval                       # finding number of time-stamps
    delta_t = t_duration / math.ceil(time_stamps)                  # creating new time intervals to allow array to reach end point
    
    t = np.arange(t_start, t_end + delta_t, delta_t)               # setting up array from initial to final time in increments of delta t
    t_mean = (t_start + t_end) / 2                                 # finding mean of time for sine-gaussian equation
    
    S = A*np.sin(2*np.pi*f*t)*np.exp((-(t - t_mean)**2)/(2*sigma)) # sine-gaussian equation
    
    return t, S

In [ ]:
def random_noise(t_start, t_end, time_interval):
    
    """
    This function takes as input the bounds of
    time and the interval of time.
    
    INPUT:
    ------
    t_start : intial time
    t_end : final time
    time_interval : interval of time between each value
    
    RETURNS:
    -------
    The time series for randomly generated noise.
    
    """
    
    